In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as con
import astropy.units as u
from astropy.cosmology import Planck15 as cosmo

## Derive Luminosity from star formation rate

The formula to be used is from Kennicutt (1998) and stated below:

$$SFR (in \ M_\odot year^{-1}) = 4.5 \times 10^{-44} \cdot L_{FIR} (in \ erg \ s^{-1})$$

$$\Rightarrow L_{FIR} (in \ erg \ s^{-1}) = \frac{SFR (in \ M_\odot year^{-1}) }{4.5 \times 10^{-44}}$$

In [2]:
sfr = 30
kappa = 4.5e-44

lfir = sfr/kappa ## in erg/sec
print('The infrared luminosity is: ', lfir)

The infrared luminosity is:  6.666666666666667e+44


Now we want to compute the CO luminosity from this IR luminosity. We can use the following line,

$$\log L_{IR} = 1.13 \cdot \log L'_{CO} + 0.53 $$

In [3]:
lsun = ((con.L_sun).value)*(1e7)
lfir_sun = lfir/lsun
logco = (np.log10(lfir_sun)-0.53)/1.13
lco_diff_units = 10**(logco)

print('The CO luminosity is: ', lco_diff_units)

The CO luminosity is:  3010961089.7614913


We now want to choose the appropriate frequency for observation.

In [4]:
lines = np.array(['CO10', 'CO21', 'CO32', 'C043', 'CO54', 'CO65', 'CO75'])

rest_frame = np.array([115.271204, 230.537990, 345.795989, 461.040770, 576.267904, 691.473090, 806.651806])

redshift = 1.036

obs_freq = rest_frame/(1+redshift)

for i in range(len(lines)):
    print(lines[i] + '\t' + str(obs_freq[i]))

CO10	56.616504911591356
CO21	113.23083988212181
CO32	169.8408590373281
C043	226.44438605108056
CO54	283.0392455795678
CO65	339.6233251473477
CO75	396.1944037328094


Hence we want to use CO(3-2). And we want to convert our $L'_{CO}$ into $L'_{3-2}$, by multiplying a factor of 0.6

In [5]:
lco_32 = lco_diff_units*0.6
print(lco_32)

1806576653.8568947


The adding lensing effect,

In [6]:
lco_32_lens = 4.3*lco_32
print(lco_32_lens)

7768279611.584647


In [7]:
dl = (cosmo.luminosity_distance(redshift)).value
freq1 = obs_freq[2]

aa = (lco_32_lens*freq1*freq1*((1+redshift)**3))
bb = dl*dl*(3.25e7)

flux_s = aa/bb

print(flux_s)

1.1561433012778473


In [8]:
flux_density = flux_s/200
print(flux_density)

0.005780716506389236
